In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

In [2]:
# If you have a CUDA enabled graphics card
# Using CUDA 12.6
print("Is CUDA available? ", torch.cuda.is_available())
print("GPU device name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

Is CUDA available?  True
GPU device name:  NVIDIA GeForce GTX 1080


In [18]:

# What is a Tensor?
# ----------------------------------------------------------------------
# | Dimensions |  Mathematical Name  |          Description             |
# ----------------------------------------------------------------------
# |      0     |       Scalar         |  Magnitude only (single value)  |
# ----------------------------------------------------------------------
# |      1     |       Vector         |  1D array (e.g., list of values)|
# ----------------------------------------------------------------------
# |      2     |       Matrix         |  2D table (e.g., grid or square)|
# ----------------------------------------------------------------------
# |      3     |      3-Tensor        |  3D table (e.g., cube of values)|
# ----------------------------------------------------------------------
# |      n     |      n-Tensor        |  Higher dimensional structures  |
# ----------------------------------------------------------------------

# Define data transforms for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((512, 512)),  # Resize images to 512x512; Makes sure they are all the same resolution

        # RandomHorizontalFlip() and RandomRotation(20) help reduce overfitting by creating variations of the same image. Better Generalization
        transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally for augmentation
        transforms.RandomRotation(20),  # Randomly rotate images by up to 20 degrees
        
        transforms.ToTensor(),  # Convert images to PyTorch tensors, this is the format used by PyTorch


        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images with mean and std deviation
    ]),
    'test': transforms.Compose([
        transforms.Resize((512, 512)),  # Resize images to 512x512; Makes sure they are all the same resolution
        transforms.ToTensor(),  # Convert images to PyTorch tensors, this is the format used by PyTorch
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images with mean and std deviation
    ]),
}

# Set directories for training and test data
data_dir = 'BrainTumorImagesDataset/'
# The datasets.ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'Training'), data_transforms['train']),
    'test': datasets.ImageFolder(os.path.join(data_dir, 'Testing'), data_transforms['test'])
}


# Create data loaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=True, num_workers=4)
}

Dataset ImageFolder
    Number of datapoints: 1311
    Root location: BrainTumorImagesDataset/Testing
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [8]:
class BrainTumorCNN(nn.Module): #nn.Module is a base class in PyTorch that provides methods for defining layers and forward pass
    def __init__(self):
        super(BrainTumorCNN, self).__init__() # Calls the parent class constructor

        
        # notice that the output filters of the previous layer == the inputs the the next layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) # 3 input channels (RGB), 32 output filters
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # 32 imput filters, 64 output filters
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1) # 64 input filters, 128 output filters
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 64 * 64, 128)  # Adjust based on image size
        self.fc2 = nn.Linear(128, 4)  # 4 output classes: pituitary, notumor, meningioma, glioma
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 64 * 64)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = BrainTumorCNN()

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# Define the device to use (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)
device

device(type='cuda')

In [15]:
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=20):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Train the model
trained_model = train_model(model, criterion, optimizer, dataloaders, device, num_epochs=20)

Epoch 1/20
----------
train Loss: 0.5002 Acc: 0.8113
test Loss: 0.4258 Acc: 0.8200
Epoch 2/20
----------
train Loss: 0.4028 Acc: 0.8524
test Loss: 0.3746 Acc: 0.8482
Epoch 3/20
----------
train Loss: 0.3378 Acc: 0.8729
test Loss: 0.3491 Acc: 0.8619
Epoch 4/20
----------
train Loss: 0.2947 Acc: 0.8911
test Loss: 0.2661 Acc: 0.8940
Epoch 5/20
----------
train Loss: 0.2543 Acc: 0.9056
test Loss: 0.2943 Acc: 0.8818
Epoch 6/20
----------
train Loss: 0.2440 Acc: 0.9116
test Loss: 0.2089 Acc: 0.9314
Epoch 7/20
----------
train Loss: 0.2240 Acc: 0.9214
test Loss: 0.2089 Acc: 0.9207
Epoch 8/20
----------
train Loss: 0.1844 Acc: 0.9321
test Loss: 0.2361 Acc: 0.9054
Epoch 9/20
----------
train Loss: 0.1763 Acc: 0.9363
test Loss: 0.1758 Acc: 0.9375
Epoch 10/20
----------
train Loss: 0.1716 Acc: 0.9414
test Loss: 0.1510 Acc: 0.9489
Epoch 11/20
----------
train Loss: 0.1569 Acc: 0.9452
test Loss: 0.1528 Acc: 0.9519
Epoch 12/20
----------
train Loss: 0.1279 Acc: 0.9520
test Loss: 0.1658 Acc: 0.9512
E